In [1]:
import pandas as pd, numpy as np, requests

dmg_spells = pd.read_html('https://listfist.com/list-of-dungeons-dragons-5e-spells-by-damage')[0]

In [2]:
dmg_spells

,L,Spell Name,Spell Save,Damage,Damage Type,Damage Increases at Higher Level,Upcast
0,0,Acid Splash,Dexterity,1d6,Acid,✓,✗
1,0,Booming Blade,✗,1d8,Thunder,✓,✗
2,0,Chill Touch,✗,1d8,Necrotic,✓,✗
3,0,Create Bonfire,Dexterity,1d8,Fire,✓,✗
4,0,Eldritch Blast,✗,1d10,Force,✓,✗
...,...,...,...,...,...,...,...
171,9,Psychic Scream,Intelligence,14d6,Psychic,✗,✗
172,9,Ravenous Void,Strength,5d10,Force,✗,✗
173,9,Storm of Vengeance,Constitution,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",✗,Dice
174,9,Time Ravage,Constitution,10d12,Necrotic,✗,✗


In [3]:
dmg_spells_copy = dmg_spells.copy()
dmg_spells_copy.loc[dmg_spells_copy['Damage Increases at Higher Level'] == '✓','Damage Increases at Higher Level'] = True
dmg_spells_copy.loc[dmg_spells_copy['Damage Increases at Higher Level'] == '✗','Damage Increases at Higher Level'] = False
dmg_spells_copy.loc[dmg_spells_copy['Upcast'] == '✗','Upcast'] = None
dmg_spells_copy.loc[dmg_spells_copy['Spell Save'] == '✗','Spell Save'] = None
dmg_spells_copy['Spell Save'] = np.select(
    [
        dmg_spells_copy['Spell Save']=='Dexterity',
        dmg_spells_copy['Spell Save']=='Constitution',
        dmg_spells_copy['Spell Save']=='Strength',
        dmg_spells_copy['Spell Save']=='Intelligence',
        dmg_spells_copy['Spell Save']=='Wisdom',
        dmg_spells_copy['Spell Save']=='Charisma'
    ],
    [
        'DEX','CON','STR','INT','WIS','CHR'
    ],
    default=None
)
dmg_spells_copy.columns = [
    'Lvl','Name','Save','Dmg','DmgType','HigherLvlIncrease','Upcast'
]
dmg_spells_copy['dmgCpy'] = dmg_spells_copy['Dmg'].copy()
# dmg_spells_copy['dmg_die']
dmg_spells_copy
# dmg_spells_copy.columns
from bs4 import BeautifulSoup as BS,re 

In [4]:
# 
# dmg_spells_copy['desc'] = ''
# descs = []
# ucs = []
# lvlInc = []
# for i,row in dmg_spells_copy.iterrows():
#     uc = None
#     out = None
#     try:
#         f = open(
#             './data_for_parsing/spells/{}.html'.format(row['Name']),
#             'r'
#         ).read()
#     except:
#         print('file not found: {}'.format(
#             './data_for_parsing/spells/{}.html'.format(row['Name'])
#         ))
#         pass
#     print(row['Name'])
#     soup = BS(f)
#     #parse the base damage...
#     body = soup(id=re.compile(r'page-content*'))[0]
#     description = re.findall('(?<=Duration:).*?(?=\<em\>)',body.decode_contents(),re.DOTALL)
#     if len(description) > 0:
#         desc = list.copy(description)
#         desc = desc[0]
#         desc = desc[desc.index('<p>'):]
#         p = re.compile(r'<.*?>')
#         desc = re.sub(p,' ',desc)
#         # desc = desc[desc.index('\n'):]
#         desc = re.sub(r'\s+', ' ', desc)
#         descs.append(desc)

#     higherLvls = re.findall(
#             '(?<=At Higher Levels\.).*?(?=\</p\>)',
#             body.decode_contents(),
#             re.DOTALL
#     )
#     deals_damage = re.findall('(?<=take|deal).*?(?=damage)',body.decode_contents())
#     print("\t{}".format(deals_damage))
#     try:
#         if row['HigherLvlIncrease']:
#             #parse the increase
#             try:
#                 chrLvlInc = re.findall('(?<=when you reach).*?(?=\n)',higherLvls[0])
#                 chrLvlInc2 = re.findall('(\d{1,2}[a-zA-Z]{2}) level \((\dd\d)\)',chrLvlInc[0])
#                 if len(chrLvlInc2) > 0:
#                     chrLvls = [int(x[0][:-2]) for x in chrLvlInc2]
#                     dmgDice = [tuple(x[1].split('d')) for x in chrLvlInc2]
#                     dmgDice = [{int(x):int(y)} for x,y in dmgDice]
#                     out = [{a:b} for a,b in zip(chrLvls,dmgDice)]
#                     if len(out) > 0:
#                         lvlInc.append(out)
#                     else:
#                         lvlInc.append(None)
#                     print("\tChar Level Increases: {}".format(out))
#             except:
#                 lvlInc.append(None)
#             finally:
#                 ucs.append(None)
#             # print(chrLvlInc2)
#         if row['Upcast']:
#             try:
#                 upcast = re.findall(
#                     re.compile(r'of \d{1,2}[a-z]{2} level or higher.*?(?=for each)'),
#                     higherLvls[0]
#                 )
#                 min_level = re.findall('(?<=of )(\d+[a-zA-Z]{2})(?= level)',upcast[0])
#                 min_level = min_level[0][:-2]
#                 inc_by = re.findall('(?<=increase).*',upcast[0])
#                 if len(inc_by) > 0:
#                     inc_by = re.findall('(?<=by ).*',inc_by[0])
#                     # inc_by = inc_by[0]
#                     inc_by = tuple(inc_by[0].split('d'))
#                     # inc_by = {inc_by[0]:inc_by[1]}
#                     # inc_by = [tuple(x)]
#                     inc_by = {int(inc_by[0]):int(inc_by[1])}
#                 # if len(inc_by) > 0:
#                 #     inc_by = inc_by[0]
#                 uc = {int(min_level):inc_by}
#                 if len(uc.keys()) > 0:
#                     ucs.append(uc)
#                 else:
#                     ucs.append(None)
#                 print("\tMin Level Increase: {}\n\tUpcast:{}\n\tInc By:{}\n\tuc:{}".format(
#                         min_level,
#                         upcast,
#                         inc_by,
#                         uc
#                     )
#                 )
#             except:
#                 ucs.append(None)
#             finally:
#                 lvlInc.append(None)
            
            
#         if row['Save']:
#             pass

#     except:
#         print('erroneous marking')
#         lvlInc.append(None)
#         ucs.append(None)
#         pass
#         #parse the upcast information

# dmg_spells_copy['desc'] = descs
# dmg_spells_copy['upcast'] = ucs
# dmg_spells_copy['lvlInc'] = lvlInc
# dmg_spells_copy

In [5]:
from bs4 import BeautifulSoup as BS,re 
dmg_spells_copy['desc'] = ''
descs = []
ucs = []
lvlInc = []
for i,row in dmg_spells_copy.iterrows():
    uc = None
    out = None
    try:
        f = open(
            './data_for_parsing/spells/{}.html'.format(row['Name']),
            'r'
        ).read()
    except:
        print('file not found: {}'.format(
            './data_for_parsing/spells/{}.html'.format(row['Name'])
        ))
        pass
    print(row['Name'])
    soup = BS(f)
    #parse the base damage...
    body = soup(id=re.compile(r'page-content*'))[0]
    description = re.findall('(?<=Duration:).*?(?=\<em\>)',body.decode_contents(),re.DOTALL)
    if len(description) > 0:
        desc = list.copy(description)
        desc = desc[0]
        desc = desc[desc.index('<p>'):]
        p = re.compile(r'<.*?>')
        desc = re.sub(p,' ',desc)
        # desc = desc[desc.index('\n'):]
        desc = re.sub(r'\s+', ' ', desc)
        descs.append(desc)

    higherLvls = re.findall(
            '(?<=At Higher Levels\.).*?(?=\</p\>)',
            body.decode_contents(),
            re.DOTALL
    )
    deals_damage = re.findall('(?<=take|deal).*?(?=damage)',body.decode_contents())
    print("\t{}".format(deals_damage))

    if row['HigherLvlIncrease']:
        #parse the increase
        try:
            chrLvlInc = re.findall('(?<=when you reach).*?(?=\n)',higherLvls[0])
            chrLvlInc2 = re.findall('(\d{1,2}[a-zA-Z]{2}) level \((\dd\d)\)',chrLvlInc[0])
            if len(chrLvlInc2) > 0:
                chrLvls = [int(x[0][:-2]) for x in chrLvlInc2]
                dmgDice = [tuple(x[1].split('d')) for x in chrLvlInc2]
                dmgDice = [{int(x):int(y)} for x,y in dmgDice]
                out = [{a:b} for a,b in zip(chrLvls,dmgDice)]
                # if len(out) > 0:
                #     lvlInc.append(out)
                # else:
                #     lvlInc.append(None)
                print("\tChar Level Increases: {}".format(out))
        except:
            out = None

    if row['Upcast']:
        try:
            upcast = re.findall(
                re.compile(r'of \d{1,2}[a-z]{2} level or higher.*?(?=for each)'),
                higherLvls[0]
            )
            min_level = re.findall('(?<=of )(\d+[a-zA-Z]{2})(?= level)',upcast[0])
            min_level = min_level[0][:-2]
            inc_by = re.findall('(?<=increase).*',upcast[0])
            if len(inc_by) > 0:
                inc_by = re.findall('(?<=by ).*',inc_by[0])
                # inc_by = inc_by[0]
                inc_by = tuple(inc_by[0].split('d'))
                # inc_by = {inc_by[0]:inc_by[1]}
                # inc_by = [tuple(x)]
                inc_by = {int(inc_by[0]):int(inc_by[1])}
            # if len(inc_by) > 0:
            #     inc_by = inc_by[0]
            uc = {int(min_level):inc_by}
            print("\tMin Level Increase: {}\n\tUpcast:{}\n\tInc By:{}\n\tuc:{}".format(
                    min_level,
                    upcast,
                    inc_by,
                    uc
                )
            )
        except:
            uc = None         
        
    if row['Save']:
        pass

    ucs.append(uc)
    lvlInc.append(out)

dmg_spells_copy['desc'] = descs
dmg_spells_copy['upcast'] = ucs
dmg_spells_copy['lvlInc'] = lvlInc
dmg_spells_copy

Acid Splash
	[' 1d6 acid ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Booming Blade
	['s 1d8 thunder ', 's an extra 1d8 thunder ', 's for moving increases to 2d8. Both ']
Chill Touch
	['s 1d8 necrotic ']
	Char Level Increases: [{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]
Create Bonfire
	[' 1d8 fire ']
	Char Level Increases: [{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]
Eldritch Blast
	['s 1d10 force ']
Fire Bolt
	['s 1d10 fire ']
Frostbite
	['s 1d6 cold ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Green-Flame Blade
	['s fire ', 's an extra 1d8 fire ']
Infestation
	['s 1d6 poison ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Lightning Lure
	[' 1d8 lightning ']
	Char Level Increases: [{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]
Magic Stone
	['s bludgeoning ']
Mind Sliver
	[' 1d6 psychic ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Poison Spray
	[' 1d12 poison ']
Primal Savagery
	['s 1d10 acid ']
Produce Flam

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,desc,upcast,lvlInc
0,0,Acid Splash,DEX,1d6,Acid,True,None,1d6,You hurl a bubble of acid. Choose one creatur...,None,"[{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]"
1,0,Booming Blade,None,1d8,Thunder,True,None,1d8,You brandish the weapon used in the spell’s c...,None,None
2,0,Chill Touch,None,1d8,Necrotic,True,None,1d8,"You create a ghostly, skeletal hand in the sp...",None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]"
3,0,Create Bonfire,DEX,1d8,Fire,True,None,1d8,You create a bonfire on ground that you can s...,None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]"
4,0,Eldritch Blast,None,1d10,Force,True,None,1d10,A beam of crackling energy streaks toward a c...,None,None
...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,14d6,Psychic,False,None,14d6,You unleash the power of your mind to blast t...,None,None
172,9,Ravenous Void,STR,5d10,Force,False,None,5d10,You create a 20-foot-radius sphere of destruc...,None,None
173,9,Storm of Vengeance,CON,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"A churning storm cloud forms, centered on a p...",None,None
174,9,Time Ravage,CON,10d12,Necrotic,False,None,10d12,You target a creature you can see within rang...,None,None


In [6]:
dmg_spells_copy['dmgCpy'] = dmg_spells_copy['Dmg'].str.findall(r'\d+d\d+')
# dmg_spells_copy['dmg_die_count'] = np.select(
#     [dmg_spells_copy['dmgCpy'].str.contains(',')],
#     [None],
#     default = dmg_spells_copy['dmgCpy'].str.split('d')[0] 
# )
# dmg_spells_copy['dmg_die_size'] = np.select(
#     [dmg_spells_copy['dmgCpy'].str.contains(',')],
#     [None],
#     default = dmg_spells_copy['dmgCpy'].str.split('d')
# )

type(dmg_spells_copy['dmgCpy'].loc[0])
d1,d2 = [],[]
for i, row in dmg_spells_copy.iterrows():
    if len(row['dmgCpy']) == 1:
        d = row['dmgCpy'][0].split('d')
        d1.append(d[0])
        d2.append(d[1])
    else:
        d1.append(None)
        d2.append(None)

dmg_spells_copy['dmg_die_count'] = d1
dmg_spells_copy['dmg_die_size'] = d2
        
dmg_spells_copy.drop(columns=['dmgCpy'],inplace=True)
dmg_spells_copy['Upcast'] = np.select(
    [~dmg_spells_copy['Upcast'].isna()],[True],default=False
)
dmg_spells_copy

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,dmg_die_count,dmg_die_size
0,0,Acid Splash,DEX,1d6,Acid,True,False,You hurl a bubble of acid. Choose one creatur...,None,"[{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]",1,6
1,0,Booming Blade,None,1d8,Thunder,True,False,You brandish the weapon used in the spell’s c...,None,None,1,8
2,0,Chill Touch,None,1d8,Necrotic,True,False,"You create a ghostly, skeletal hand in the sp...",None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]",1,8
3,0,Create Bonfire,DEX,1d8,Fire,True,False,You create a bonfire on ground that you can s...,None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]",1,8
4,0,Eldritch Blast,None,1d10,Force,True,False,A beam of crackling energy streaks toward a c...,None,None,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,14d6,Psychic,False,False,You unleash the power of your mind to blast t...,None,None,14,6
172,9,Ravenous Void,STR,5d10,Force,False,False,You create a 20-foot-radius sphere of destruc...,None,None,5,10
173,9,Storm of Vengeance,CON,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,True,"A churning storm cloud forms, centered on a p...",None,None,None,None
174,9,Time Ravage,CON,10d12,Necrotic,False,False,You target a creature you can see within rang...,None,None,10,12


In [7]:
dmg_spells_copy.loc[(dmg_spells_copy['HigherLvlIncrease']==True) &(dmg_spells_copy['lvlInc'].isna())]

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,dmg_die_count,dmg_die_size
1,0,Booming Blade,None,1d8,Thunder,True,False,You brandish the weapon used in the spell’s c...,None,None,1,8
4,0,Eldritch Blast,None,1d10,Force,True,False,A beam of crackling energy streaks toward a c...,None,None,1,10
5,0,Fire Bolt,None,1d10,Fire,True,False,You hurl a mote of fire at a creature or obje...,None,None,1,10
7,0,Green-Flame Blade,None,1d8,Fire,True,False,You brandish the weapon used in the spell’s c...,None,None,1,8
10,0,Magic Stone,None,1d6+SAM,Bludgeoning,True,False,You touch one to three pebbles and imbue them...,None,None,1,6
12,0,Poison Spray,CON,1d12,Poison,True,False,You extend your hand toward a creature you ca...,None,None,1,12
13,0,Primal Savagery,None,1d10,Acid,True,False,You channel primal magic to cause your teeth ...,None,None,1,10
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,False,You point at one creature you can see within ...,None,None,None,None


In [8]:
dmg_spells_copy.loc[(dmg_spells_copy['Upcast']==True) &(dmg_spells_copy['upcast'].isna())]

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,dmg_die_count,dmg_die_size
27,1,Armor of Agathys,None,5,Cold,False,True,"A protective magical force surrounds you, man...",None,None,None,None
30,1,Catapult,DEX,3d8,Bludgeoning,False,True,Choose one object weighing 1 to 5 pounds with...,None,None,3,8
41,1,Hex,None,1d6,Necrotic,False,True,You place a curse on a creature that you can ...,None,None,1,6
42,1,Hunter's Mark,None,1d6,Variable,False,True,You choose a creature you can see within rang...,None,None,1,6
50,1,Tasha's Caustic Brew,DEX,2d4,Acid,False,True,A stream of acid emanates from you in a line ...,None,None,2,4
59,2,Cordon of Arrows,DEX,1d6,Piercing,False,True,You plant four pieces of nonmagical ammunitio...,None,None,1,6
62,2,Flame Blade,None,3d6,Fire,False,True,You evoke a fiery blade in your free hand. Th...,None,None,3,6
72,2,Shadow Blade,None,2d8,Psychic,False,True,You weave together threads of shadow to creat...,None,None,2,8
75,2,Spiritual Weapon,None,1d8+SAM,Force,False,True,"You create a floating, spectral weapon within...",None,None,1,8
78,3,Ashardalon's Stride,None,1d6,Fire,False,True,The billowing flames of a dragon blast from y...,None,None,1,6


In [9]:
dmg_spells_copy.loc[(dmg_spells_copy['dmg_die_count'].isna())]

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,dmg_die_count,dmg_die_size
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,False,You point at one creature you can see within ...,None,None,None,None
27,1,Armor of Agathys,None,5,Cold,False,True,"A protective magical force surrounds you, man...",None,None,None,None
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,True,"You hurl an undulating, warbling mass of chao...",{2: []},None,None,None
43,1,Ice Knife,DEX,1d10 / 2d6,"Cold, Piercing",False,True,You create a shard of ice and fling it at one...,{2: {1: 6}},None,None,None
49,1,Searing Smite,CON,1d6 / 1d6,Fire,False,True,The next time you hit a creature with a melee...,{2: {1: 6}},None,None,None
65,2,Maximilian's Earthen Grasp,STR,2d6 / 2d6,Bludgeoning,False,False,"Choose a manufactured metal object, such as a...",None,None,None,None
66,2,Melf's Acid Arrow,None,4d4 / 2d4,Acid,False,True,A shimmering green arrow streaks toward a tar...,{3: {1: 4}},None,None,None
87,3,Hunger of Hadar,DEX,2d6 / 2d6,"Acid, Cold",False,False,You open a gateway to the dark between the st...,None,None,None,None
89,3,Lightning Arrow,DEX,4d8 / 2d8,Lightning,False,True,The next time you make a ranged weapon attack...,{4: {1: 8}},None,None,None
105,4,Guardian of Faith,DEX,20,Radiant,False,False,A Large spectral guardian appears and hovers ...,None,None,None,None


In [12]:
dmg_spells_copy[~dmg_spells_copy['upcast'].isna()].head(30)

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,dmg_die_count,dmg_die_size
26,1,Absorb Elements,None,1d6,Variable,False,True,The spell captures some of the incoming energ...,{2: {1: 6}},None,1,6
28,1,Arms of Hadar,STR,2d6,Necrotic,False,True,"You invoke the power of Hadar, the Dark Hunge...",{2: {1: 6}},None,2,6
29,1,Burning Hands,DEX,3d6,Fire,False,True,As you hold your hands with thumbs touching a...,{2: {1: 6}},None,3,6
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,True,"You hurl an undulating, warbling mass of chao...",{2: []},None,None,None
32,1,Chromatic Orb,None,3d8,Variable,False,True,You hurl a 4-inch-diameter sphere of energy a...,{2: {1: 8}},None,3,8
33,1,Dissonant Whispers,WIS,3d6,Psychic,False,True,You whisper a discordant melody that only one...,{2: {1: 6}},None,3,6
35,1,Earth Tremor,DEX,1d6,Bludgeoning,False,True,You cause a tremor in the ground in a 10-foot...,{2: {1: 6}},None,1,6
36,1,Ensnaring Strike,STR,1d6,Piercing,False,True,The next time you hit a creature with a weapo...,{2: {1: 6}},None,1,6
37,1,Frost Fingers,CON,2d8,Cold,False,True,Freezing cold blasts from your fingertips in ...,{2: {1: 8}},None,2,8
38,1,Guiding Bolt,None,4d6,Radiant,False,True,A flash of light streaks toward a creature of...,{2: {1: 6}},None,4,6


In [13]:
dmg_spells_copy.loc[dmg_spells_copy['dmg_die_count'].isna()]

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,dmg_die_count,dmg_die_size
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,False,You point at one creature you can see within ...,None,None,None,None
27,1,Armor of Agathys,None,5,Cold,False,True,"A protective magical force surrounds you, man...",None,None,None,None
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,True,"You hurl an undulating, warbling mass of chao...",{2: []},None,None,None
43,1,Ice Knife,DEX,1d10 / 2d6,"Cold, Piercing",False,True,You create a shard of ice and fling it at one...,{2: {1: 6}},None,None,None
49,1,Searing Smite,CON,1d6 / 1d6,Fire,False,True,The next time you hit a creature with a melee...,{2: {1: 6}},None,None,None
65,2,Maximilian's Earthen Grasp,STR,2d6 / 2d6,Bludgeoning,False,False,"Choose a manufactured metal object, such as a...",None,None,None,None
66,2,Melf's Acid Arrow,None,4d4 / 2d4,Acid,False,True,A shimmering green arrow streaks toward a tar...,{3: {1: 4}},None,None,None
87,3,Hunger of Hadar,DEX,2d6 / 2d6,"Acid, Cold",False,False,You open a gateway to the dark between the st...,None,None,None,None
89,3,Lightning Arrow,DEX,4d8 / 2d8,Lightning,False,True,The next time you make a ranged weapon attack...,{4: {1: 8}},None,None,None
105,4,Guardian of Faith,DEX,20,Radiant,False,False,A Large spectral guardian appears and hovers ...,None,None,None,None


column = [
    {CHAR LEVEL:[
        {1: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
        ]
        },
        {5: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
            ]
        },
        {11: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
            ]
        },
        {17: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
            ]
        }
    },

    {UPCAST:
    {
    Cold : {1 : 6},
    Piercing : {2 : 8}
    }

############################################################

DMG = [
        {
            1:[
                {Cold : {1 : 6}}, 
                {Piercing : {2 : 8}}
            ]
        }
        {
            5: [
                {Cold : {2 : 6}}, 
                {Piercing : {2 : 8}}
            ]
        },
        {
            11: [
                {Cold : {3 : 6}}, 
                {Piercing : {3: 8}}
            ]
        },
        {
            17: [
                {Cold : {4 : 6}}, 
                {Piercing : {4 : 8}}
            ]
        }
    }
]

UPCAST:
    [
        {DMG_TYPE:{min_level:{dice qty: die size}}},
        {DMG_TYPE:{min_level:{dice qty: die size}}}
    ]